In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Defining the Networking class

In [3]:
# all models in PyTorch should be derived from nn.Module

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
                                                            #  Assume N is the batch size
        # Block 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)         # (N, 1, 28, 28 )   -> (N, 32, 28, 28) , RF: 3 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)        # (N, 32, 28, 28)   -> (N, 64, 28, 28) , RF: 5
        self.pool1 = nn.MaxPool2d(2, 2)                     # (N, 64, 28, 28)   -> (N, 64, 14, 14) , RF: 6
        
        # Block 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)       # (N, 64, 14, 14)   -> (N, 128, 14, 14), RF: 10                
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)      # (N, 128, 14, 14)  -> (N, 256, 14, 14), RF: 14
        self.pool2 = nn.MaxPool2d(2, 2)                     # (N, 256, 14, 14)  -> (N, 256, 7, 7)  , RF: 16
        
        # Block 3
        self.conv5 = nn.Conv2d(256, 512, 3)                 # (N, 256, 7, 7)    -> (N, 512, 5, 5)  , RF: 24      
        self.conv6 = nn.Conv2d(512, 1024, 3)                # (N, 512, 7, 7)    -> (N, 1024,3,3)   , RF: 32
        self.conv7 = nn.Conv2d(1024, 10, 3)                 # (N, 1024,7,7)     -> (N, 10, 1,1)    , RF: 40
        
    def forward(self, x):
        
        # Block 1 (CONV -> RELU -> CONV -> RELU -> MP)
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   
        
        # Block 2 (CONV -> RELU -> CONV -> RELU -> MP)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))

        # Block 3 (CONV -> RELU -> CONV -> RELU)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))

        # Final Layer (CONV)
        x = self.conv7(x)
        
        # Flatten and Reshaping it to 10 output (Vector)
        x = x.view(-1, 10)
        
        # Applying Softmax on the vector to get the intensity of 10 different outputs 
        return F.log_softmax(x)

## Checking the Network using `torchsummary`

In [4]:
!pip install torchsummary

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [5]:
# use torchsummary to print a summary of the above model
from torchsummary import summary

# check if a GPU is available (if a GPU is available, CUDA would be loaded)
use_cuda = torch.cuda.is_available()

# get a reference to the available GPU instance
device = torch.device("cuda" if use_cuda else "cpu")

# send the model to the GPU device
model = Net().to(device)

# print a summary of the above model, given the input_size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Loading the dataset

In [6]:
# seed of all random processes within the torch instance
torch.manual_seed(1)
# number of input data points in each batch
batch_size = 128

# pin_memory: to use non-paged CPU memory to allow for async data xfer to GPU mem
# num_workers: keep 1 thread to preprocess the next batch of input
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# instantiate a DataLoader for training
train_loader = torch.utils.data.DataLoader(
    # download MNIST dataset, and only the Training dataset
    datasets.MNIST('../data', train=True, download=True,
                    # pre-processing that each chunk of input data will go through
                    transform=transforms.Compose([
                        # convert to a Tensor
                        transforms.ToTensor(),
                        # data in the tensor will be normalized to the below
                        # mentioned Mean and Standard Deviation
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    # randomly shuffle input data for each batch
    batch_size=batch_size, shuffle=True, **kwargs)

# instantiate a DataLoader for testing
test_loader = torch.utils.data.DataLoader(
    # download MNIST dataset, and only the Testing dataset this time
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        # convert to a Tensor
                        transforms.ToTensor(),
                        # use the same mu and sigma values as for training data
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    # randomly shuffle input data for each batch
    batch_size=batch_size, shuffle=True, **kwargs)

## Defining the Train and Test wrapper function

In [7]:
# Basic workflows for training and testing of the model

# library to help show the progress bar
from tqdm import tqdm

# training workflow
def train(model, device, train_loader, optimizer, epoch):
    # tell the model that we are in the training phase
    # this helps Batch Norm and Dropout modules to take effect
    model.train()

    # prepare to iterate the progress bar over the input training data loader
    pbar = tqdm(train_loader)

    # standard 5-step process of training, for each iteration of input data batch
    for batch_idx, (data, target) in enumerate(pbar):
        # initialize the training input data and labels to the device memory
        data, target = data.to(device), target.to(device)
    
        # Step-1: intialize the optimizer parameters so that gradients of other
        # epochs dont mess with the current epoch
        optimizer.zero_grad()
        # Step-2: do a forward pass through the model using the current data batch
        output = model(data)
        # Step-3: compute the loss after the forward pass
        loss = F.nll_loss(output, target)
        # Step-4: compute the gradients for all tensors in the model using backpropagation
        loss.backward()
        # Step-5: iterate over all parameters (tensors) it is supposed to update
        # and use their internally stored grad to update their values
        optimizer.step()
    
        # visual output of the progress
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

# testing workflow
def test(model, device, test_loader):
    # tell the model that we are in the testing phase
    model.eval()
    test_loss = 0
    correct = 0

    # tell PyTorch not to update the gradients while doing the following
    with torch.no_grad():
        # iterate over the test Data Loader, with one data batch every iteration
        for data, target in test_loader:
            # initialize the testing input data and labels to the device memory
            data, target = data.to(device), target.to(device)
            # run the test data through the model
            output = model(data)
            # compute the loss and accumulate the loss over all batches of test data
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            # identify the output prediction as the index with the highest
            # log-probability
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # if the prediction is the same as the expected (target),
            # track it in the "correct" counter
            correct += pred.eq(target.view_as(pred)).sum().item()

    # compute the average loss over the test dataset
    test_loss /= len(test_loader.dataset)

    # print a summary of the test run
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Starting the Training

In [8]:
# Basic driver code to train and test the model

# send the model to the identified device (GPU)
model = Net().to(device)
# setup the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# multi-epoch train-test loop (currently shows only 1 epoch)
for epoch in range(1, 2):
    # train the model, results in model parameters to be adjusted
    train(model, device, train_loader, optimizer, epoch)
    # test the model accuracy with the learnt model parameters
    test(model, device, test_loader)

loss=0.03632986545562744 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.39it/s] 



Test set: Average loss: 0.0659, Accuracy: 9787/10000 (98%)

